In [1]:
import gdown
import os
import numpy as np
import pandas as pd
from src.data import WORKSPACE
from src.model import phoBert_CNN
import lightning as L
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint


/home/arch/.pyenv/versions/3.11.9/envs/evo/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
workspace = WORKSPACE()

VnCoreNLP model folder /home/arch/projects/college_assignments/CS313_bigdata/HateSpeechDetection/dataset/vncorenlp already exists! Please load VnCoreNLP from this folder!
2024-05-20 15:01:40 INFO  WordSegmenter:24 - Loading Word Segmentation model
/home/arch/projects/college_assignments/CS313_bigdata/HateSpeechDetection


In [3]:
train_dataloader = workspace.get_dataloader(stage="train", batch_size=4, shuffle=True, num_workers=4)
val_dataloader = workspace.get_dataloader(stage="val", batch_size=4, shuffle=False, num_workers=4)

/home/arch/.pyenv/versions/3.11.9/envs/evo/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/arch/.pyenv/versions/3.11.9/envs/evo/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
checkpoint_callback = ModelCheckpoint(
    monitor="valid_loss",
    mode="min",
    save_top_k=3,
    save_last=True,
    save_weights_only=True,
    dirpath="ckpts",
)
trainer = L.Trainer(
    max_epochs=5,
    callbacks=[checkpoint_callback]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
model = phoBert_CNN(lr = 1e-5)

/home/arch/.pyenv/versions/3.11.9/envs/evo/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/arch/.pyenv/versions/3.11.9/envs/evo/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


checkpoints are saved the the ckpts folder

In [6]:
trainer.fit(model, train_dataloader, val_dataloader)

2024-05-20 15:01:45.773121: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 15:01:46.752783: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params
-----------------------------------------------
0 | phobert  | RobertaModel      | 134 M 
1 | fc_input | Linear            | 590 K 
2 | convs    | ModuleList        | 270 K 
3 | fc       | Linear            | 387   
4 | dropout  | Dropout           | 0     
5 | valid_f1 | MulticlassF1Score | 0     
-----------------------------------------------
135 M     Trainable params
0         Non-trainable params
135 M     Total params
543.439   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

/home/arch/.pyenv/versions/3.11.9/envs/evo/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
